In [5]:
import pandas as pd
import requests
import json
from pprint import pprint

In [2]:
#base url
base_url = 'https://fantasy.premierleague.com/api/'

r = requests.get(base_url +'bootstrap-static/').json()

In [9]:
# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 10997846}


# Player data

The elements field contains data for each premier league player in the current season of FPL. We can access these data just like we would access data associated with a key in a dictionary. The response is a list of more dictionaries — one per player.

Let’s get the elements data and then show the information about the first player in the list:

In [13]:
players = r['elements']
pprint((players[4]))

In [16]:
pd.set_option('display.max_columns', None)

#create players dataframe 
players = pd.json_normalize(r['elements'])

In [18]:
players[['id', 'web_name', 'team', 'element_type']].head()

,id,web_name,team,element_type
0,1,Cédric,1,2
1,3,Xhaka,1,3
2,4,Elneny,1,3
3,5,Holding,1,2
4,6,Partey,1,3


# Supporting data 

We can get the names and strength ratings for teams by extracting the teams field from the base response into a dataframe:

In [19]:
#create teams dataframe 

teams = pd.json_normalize(r['teams'])

In [20]:
teams

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1220,1270,1240,1250,1200,1270,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,3,None,False,0,1090,1100,1110,1130,1090,1110,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,3,None,False,0,1060,1090,1070,1130,1050,1080,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,3,None,False,0,1100,1130,1100,1110,1130,1160,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,3,None,False,0,1150,1160,1140,1160,1170,1190,131
5,8,0,None,6,0,Chelsea,0,0,0,CHE,4,None,False,0,1200,1220,1200,1210,1180,1210,4
6,31,0,None,7,0,Crystal Palace,0,0,0,CRY,3,None,False,0,1100,1140,1100,1170,1080,1140,6
7,11,0,None,8,0,Everton,0,0,0,EVE,3,None,False,0,1080,1100,1080,1080,1080,1080,7
8,54,0,None,9,0,Fulham,0,0,0,FUL,3,None,False,0,1075,1100,1070,1070,1100,1130,34
9,13,0,None,10,0,Leicester,0,0,0,LEI,3,None,False,0,1130,1130,1080,1160,1180,1120,26


# Player Positions 

In [23]:
positions = pd.json_normalize(r['element_types'])

In [24]:
positions

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],72
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],236
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],288
3,4,Forwards,FWD,Forward,FWD,3,1,3,False,[],79


# Combining all data to get final player dataframe

In [25]:
# join players to teams
df = pd.merge(
    left=players,
    right=teams,
    left_on='team',
    right_on='id'
)

In [26]:
df

,chance_of_playing_next_round,chance_of_playing_this_round,code_x,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form_x,id_x,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,code_y,draw,form_y,id_y,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,100.0,100.0,58822,0,0,-3,3,0,2,0.5,0.5,0,Cédric,0.0,1,False,,2022-09-16T13:30:06.551108Z,42,58822.jpg,1.0,Alves Soares,0.1,False,None,a,1,3,2,6437,4,26365,9,0.0,0.5,Cédric,28,0,0,0,0,0,0,0,0,0,0,0,5,2.4,1.6,0.0,0.4,0,0.00000,0.00979,0.00979,0.04350,436,161,395,143,492,199,442,162,NaN,,NaN,,NaN,,0.00000,0.0,0.03147,0.03147,0.13982,0.00,549,158,386,151,390,146,400,148,0.00000,0.00000,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1220,1270,1240,1250,1200,1270,1
1,NaN,NaN,84450,0,0,1,-1,1,3,3.0,3.0,0,Granit,2.5,3,False,,None,51,84450.jpg,4.5,Xhaka,3.6,False,None,a,1,3,67,778466,464,565140,614,0.5,13.1,Xhaka,1256,3,4,6,12,0,0,0,2,0,0,7,275,323.6,310.4,251.0,88.5,15,1.56150,1.56051,3.12201,12.28030,52,19,33,27,61,37,34,23,NaN,,3.0,,NaN,,0.11189,0.0,0.11182,0.22371,0.87996,0.86,172,99,77,32,37,17,86,26,1.07484,0.42994,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1220,1270,1240,1250,1200,1270,1
2,100.0,100.0,153256,0,0,-3,3,1,3,1.0,1.0,0,Mohamed,0.5,4,False,,2022-08-30T11:30:06.278675Z,42,153256.jpg,1.2,Elneny,0.9,False,None,a,1,3,5,182885,255,228170,187,0.1,1.2,Elneny,95,0,0,0,1,0,0,0,0,0,0,0,24,4.2,5.3,0.0,1.0,1,0.00000,0.04431,0.04431,0.83280,426,189,375,178,497,195,426,191,NaN,,NaN,,NaN,,0.00000,0.0,0.04198,0.04198,0.78897,0.95,556,286,192,94,358,156,185,57,0.94737,0.00000,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1220,1270,1240,1250,1200,1270,1
3,NaN,NaN,156074,0,0,-3,3,0,2,0.5,0.5,0,Rob,0.0,5,False,,None,42,156074.jpg,1.0,Holding,0.1,False,None,a,1,3,4,6493,8,16769,13,0.0,1.0,Holding,12,0,0,0,0,0,0,0,0,0,0,0,14,2.4,0.1,0.0,0.2,0,0.00000,0.00022,0.00022,0.13740,438,163,437,165,498,203,449,165,NaN,,NaN,,NaN,,0.00000,0.0,0.00165,0.00165,1.03050,0.00,557,164,391,155,393,149,419,157,0.00000,0.00000,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1220,1270,1240,1250,1200,1270,1
4,100.0,100.0,167199,0,0,-2,2,0,3,1.5,1.5,0,Thomas,1.0,6,False,,2022-09-25T09:00:06.484502Z,48,167199.jpg,3.7,Partey,0.5,False,None,a,1,3,44,89756,76,107972,101,0.2,9.2,Partey,1049,2,0,7,7,0,0,0,1,0,0,4,205,229.2,147.9,114.0,49.2,12,1.09890,0.64933,1.74823,10.13420,117,51,116,80,158,103,120,76,NaN,,NaN,,NaN,,0.09428,0.0,0.05571,0.14999,0.86947,0.60,260,157,139,64,85,42,242,80,1.02955,0.60057,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1220,1270,1240,1250,1200,1270,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.